# DrugBank Data Extraction - Clean Version

**Purpose:** Extract DrugBank XML data for the Supplement Recommender Knowledge Graph

**What this notebook does:**
1. Extracts all required data from DrugBank XML
2. Automatically cleans and deduplicates
3. Saves final clean CSV files (no intermediate files)

**Output:** 10 clean CSV files ready for Step 2 (Normalization)

---

## 1. Setup and Configuration

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded successfully")

✓ Libraries loaded successfully


In [2]:
# ============================================================
# CONFIGURATION - Update this path to your DrugBank XML file
# ============================================================

XML_FILE = "drugbank.xml"  # UPDATE THIS PATH

# Output directory for clean CSV files
OUTPUT_DIR = Path("drugbank_tables")
OUTPUT_DIR.mkdir(exist_ok=True)

# DrugBank XML namespace
NS = {'db': 'http://www.drugbank.ca'}

print(f"✓ Configuration complete")
print(f"✓ Output directory: {OUTPUT_DIR.absolute()}")

✓ Configuration complete
✓ Output directory: /Users/jovannafernando/Desktop/Capstone/drugbank_tables


## 2. XML Parser Function

In [3]:
def iter_drugs(xml_file):
    """
    Memory-efficient XML parser that yields one drug element at a time.
    Prevents loading the entire XML file into memory.
    """
    context = ET.iterparse(xml_file, events=('start', 'end'))
    context = iter(context)
    event, root = next(context)
    
    for event, elem in context:
        if event == 'end' and elem.tag == '{http://www.drugbank.ca}drug':
            yield elem
            root.clear()

print("✓ Parser function defined")

✓ Parser function defined


## 3. Data Extraction Functions

These functions extract data and automatically deduplicate before saving.

In [4]:
def extract_drug_identifiers(xml_file):
    """
    Extract: drugs, accession_numbers, synonyms, salts
    Auto-cleans and saves 4 CSV files
    """
    print("\n" + "="*60)
    print("EXTRACTING: Drug Identifiers")
    print("="*60)
    
    drugs_data = []
    accession_data = []
    synonyms_data = []
    salts_data = []
    
    for drug in tqdm(iter_drugs(xml_file), desc="Parsing drugs"):
        # Get primary DrugBank ID
        primary_id = drug.find('db:drugbank-id[@primary="true"]', NS)
        if primary_id is None:
            primary_id = drug.find('db:drugbank-id', NS)
        if primary_id is None:
            continue
            
        drug_id = primary_id.text
        
        # Extract basic drug info
        name_elem = drug.find('db:name', NS)
        unii_elem = drug.find('db:unii', NS)
        cas_elem = drug.find('db:cas-number', NS)
        desc_elem = drug.find('db:description', NS)
        indication_elem = drug.find('db:indication', NS)
        
        drugs_data.append({
            'drugbank_id': drug_id,
            'name': name_elem.text if name_elem is not None else None,
            'unii': unii_elem.text if unii_elem is not None else None,
            'cas_number': cas_elem.text if cas_elem is not None else None,
            'description': desc_elem.text if desc_elem is not None else None,
            'indication': indication_elem.text if indication_elem is not None else None,
            'type': drug.get('type')
        })
        
        # Extract accession numbers
        for acc_num in drug.findall('db:drugbank-id', NS):
            accession_data.append({
                'drugbank_id': drug_id,
                'accession_number': acc_num.text,
                'is_primary': acc_num.get('primary') == 'true'
            })
        
        # Extract synonyms
        synonyms_elem = drug.find('db:synonyms', NS)
        if synonyms_elem is not None:
            for syn in synonyms_elem.findall('db:synonym', NS):
                if syn.text:
                    synonyms_data.append({
                        'drugbank_id': drug_id,
                        'synonym': syn.text,
                        'language': syn.get('language', 'en')
                    })
        
        # Extract salts
        salts_elem = drug.find('db:salts', NS)
        if salts_elem is not None:
            for salt in salts_elem.findall('db:salt', NS):
                salt_id = salt.find('db:drugbank-id', NS)
                salt_name = salt.find('db:name', NS)
                salt_unii = salt.find('db:unii', NS)
                
                if salt_id is not None:
                    salts_data.append({
                        'parent_drugbank_id': drug_id,
                        'salt_drugbank_id': salt_id.text,
                        'salt_name': salt_name.text if salt_name is not None else None,
                        'salt_unii': salt_unii.text if salt_unii is not None else None
                    })
    
    # Create DataFrames and clean
    print("\nCleaning and deduplicating...")
    df_drugs = pd.DataFrame(drugs_data).drop_duplicates(subset=['drugbank_id'], keep='first')
    df_accessions = pd.DataFrame(accession_data).drop_duplicates()
    df_synonyms = pd.DataFrame(synonyms_data).drop_duplicates()
    df_salts = pd.DataFrame(salts_data).drop_duplicates()
    
    # Save
    df_drugs.to_csv(OUTPUT_DIR / 'drugs.csv', index=False)
    df_accessions.to_csv(OUTPUT_DIR / 'accession_numbers.csv', index=False)
    df_synonyms.to_csv(OUTPUT_DIR / 'drug_synonyms.csv', index=False)
    df_salts.to_csv(OUTPUT_DIR / 'salts.csv', index=False)
    
    print(f"\n✓ drugs.csv: {len(df_drugs):,} drugs")
    print(f"  - With UNII: {df_drugs['unii'].notna().sum():,}")
    print(f"  - Without UNII: {df_drugs['unii'].isna().sum():,}")
    print(f"✓ accession_numbers.csv: {len(df_accessions):,} records")
    print(f"✓ drug_synonyms.csv: {len(df_synonyms):,} synonyms")
    print(f"✓ salts.csv: {len(df_salts):,} salt forms")
    
    return df_drugs, df_accessions, df_synonyms, df_salts

In [5]:
def extract_interactions(xml_file):
    """
    Extract: drug-drug interactions, food interactions
    Auto-cleans and saves 2 CSV files
    """
    print("\n" + "="*60)
    print("EXTRACTING: Drug & Food Interactions")
    print("="*60)
    
    drug_interactions = []
    food_interactions = []
    
    for drug in tqdm(iter_drugs(xml_file), desc="Parsing interactions"):
        primary_id = drug.find('db:drugbank-id[@primary="true"]', NS)
        if primary_id is None:
            primary_id = drug.find('db:drugbank-id', NS)
        if primary_id is None:
            continue
            
        drug_id = primary_id.text
        
        # Drug-drug interactions
        interactions_elem = drug.find('db:drug-interactions', NS)
        if interactions_elem is not None:
            for interaction in interactions_elem.findall('db:drug-interaction', NS):
                interacting_drug_id = interaction.find('db:drugbank-id', NS)
                interacting_drug_name = interaction.find('db:name', NS)
                description = interaction.find('db:description', NS)
                
                drug_interactions.append({
                    'drug_id': drug_id,
                    'interacting_drug_id': interacting_drug_id.text if interacting_drug_id is not None else None,
                    'interacting_drug_name': interacting_drug_name.text if interacting_drug_name is not None else None,
                    'description': description.text if description is not None else None
                })
        
        # Food interactions
        food_elem = drug.find('db:food-interactions', NS)
        if food_elem is not None:
            for food in food_elem.findall('db:food-interaction', NS):
                if food.text:
                    food_interactions.append({
                        'drug_id': drug_id,
                        'food_interaction': food.text
                    })
    
    # Create DataFrames and clean
    print("\nCleaning and deduplicating...")
    df_drug_interactions = pd.DataFrame(drug_interactions).drop_duplicates()
    df_food_interactions = pd.DataFrame(food_interactions).drop_duplicates()
    
    # Save
    df_drug_interactions.to_csv(OUTPUT_DIR / 'drug_interactions.csv', index=False)
    df_food_interactions.to_csv(OUTPUT_DIR / 'food_interactions.csv', index=False)
    
    print(f"\n✓ drug_interactions.csv: {len(df_drug_interactions):,} interactions")
    print(f"✓ food_interactions.csv: {len(df_food_interactions):,} interactions")
    
    return df_drug_interactions, df_food_interactions

In [6]:
def extract_categories(xml_file):
    """
    Extract: drug categories/classifications
    Auto-cleans and saves 1 CSV file
    """
    print("\n" + "="*60)
    print("EXTRACTING: Drug Categories")
    print("="*60)
    
    categories_data = []
    
    for drug in tqdm(iter_drugs(xml_file), desc="Parsing categories"):
        primary_id = drug.find('db:drugbank-id[@primary="true"]', NS)
        if primary_id is None:
            primary_id = drug.find('db:drugbank-id', NS)
        if primary_id is None:
            continue
            
        drug_id = primary_id.text
        
        categories_elem = drug.find('db:categories', NS)
        if categories_elem is not None:
            for category in categories_elem.findall('db:category', NS):
                category_name = category.find('db:category', NS)
                mesh_id = category.find('db:mesh-id', NS)
                
                categories_data.append({
                    'drug_id': drug_id,
                    'category': category_name.text if category_name is not None else None,
                    'mesh_id': mesh_id.text if mesh_id is not None else None
                })
    
    # Create DataFrame and clean
    print("\nCleaning and deduplicating...")
    df_categories = pd.DataFrame(categories_data).drop_duplicates()
    
    # Save
    df_categories.to_csv(OUTPUT_DIR / 'drug_categories.csv', index=False)
    
    print(f"\n✓ drug_categories.csv: {len(df_categories):,} category assignments")
    
    return df_categories

In [7]:
def extract_products_and_ingredients(xml_file):
    """
    Extract: drug products and their active ingredients
    Auto-cleans and saves 2 CSV files
    """
    print("\n" + "="*60)
    print("EXTRACTING: Products & Ingredients")
    print("="*60)
    
    products_data = []
    ingredients_data = []
    
    for drug in tqdm(iter_drugs(xml_file), desc="Parsing products"):
        primary_id = drug.find('db:drugbank-id[@primary="true"]', NS)
        if primary_id is None:
            primary_id = drug.find('db:drugbank-id', NS)
        if primary_id is None:
            continue
            
        drug_id = primary_id.text
        drug_name_elem = drug.find('db:name', NS)
        drug_name = drug_name_elem.text if drug_name_elem is not None else None
        
        products_elem = drug.find('db:products', NS)
        if products_elem is not None:
            for product in products_elem.findall('db:product', NS):
                name = product.find('db:name', NS)
                labeller = product.find('db:labeller', NS)
                ndc_id = product.find('db:ndc-id', NS)
                ndc_code = product.find('db:ndc-product-code', NS)
                dosage_form = product.find('db:dosage-form', NS)
                strength = product.find('db:strength', NS)
                route = product.find('db:route', NS)
                approved = product.find('db:approved', NS)
                country = product.find('db:country', NS)
                
                product_name_text = name.text if name is not None else None
                ndc_code_text = ndc_code.text if ndc_code is not None else None
                
                # Product data
                products_data.append({
                    'drug_id': drug_id,
                    'product_name': product_name_text,
                    'labeller': labeller.text if labeller is not None else None,
                    'ndc_id': ndc_id.text if ndc_id is not None else None,
                    'ndc_product_code': ndc_code_text,
                    'dosage_form': dosage_form.text if dosage_form is not None else None,
                    'strength': strength.text if strength is not None else None,
                    'route': route.text if route is not None else None,
                    'approved': approved.text if approved is not None else None,
                    'country': country.text if country is not None else None
                })
                
                # Ingredient data (product → active ingredient)
                ingredients_data.append({
                    'product_name': product_name_text,
                    'ndc_product_code': ndc_code_text,
                    'active_ingredient_id': drug_id,
                    'active_ingredient_name': drug_name,
                    'strength': strength.text if strength is not None else None
                })
    
    # Create DataFrames and clean
    print("\nCleaning and deduplicating...")
    df_products = pd.DataFrame(products_data).drop_duplicates()
    df_ingredients = pd.DataFrame(ingredients_data).drop_duplicates()
    
    # Save
    df_products.to_csv(OUTPUT_DIR / 'drug_products.csv', index=False)
    df_ingredients.to_csv(OUTPUT_DIR / 'product_ingredients.csv', index=False)
    
    print(f"\n✓ drug_products.csv: {len(df_products):,} products")
    print(f"✓ product_ingredients.csv: {len(df_ingredients):,} ingredient relationships")
    
    return df_products, df_ingredients

In [8]:
def extract_toxicity_contraindications(xml_file):
    """
    Extract: toxicity and contraindication info
    Auto-cleans and saves 1 CSV file
    """
    print("\n" + "="*60)
    print("EXTRACTING: Toxicity & Contraindications")
    print("="*60)
    
    toxicity_data = []
    
    for drug in tqdm(iter_drugs(xml_file), desc="Parsing toxicity"):
        primary_id = drug.find('db:drugbank-id[@primary="true"]', NS)
        if primary_id is None:
            primary_id = drug.find('db:drugbank-id', NS)
        if primary_id is None:
            continue
            
        drug_id = primary_id.text
        toxicity_elem = drug.find('db:toxicity', NS)
        indication_elem = drug.find('db:indication', NS)
        
        toxicity_data.append({
            'drug_id': drug_id,
            'toxicity': toxicity_elem.text if toxicity_elem is not None else None,
            'indication_text': indication_elem.text if indication_elem is not None else None
        })
    
    # Create DataFrame and clean
    print("\nCleaning and deduplicating...")
    df_toxicity = pd.DataFrame(toxicity_data).drop_duplicates()
    
    # Save
    df_toxicity.to_csv(OUTPUT_DIR / 'toxicity_contraindications.csv', index=False)
    
    print(f"\n✓ toxicity_contraindications.csv: {len(df_toxicity):,} records")
    print(f"  - With toxicity info: {df_toxicity['toxicity'].notna().sum():,}")
    
    return df_toxicity

## 4. Run All Extractions

**This cell runs everything.** It will take several minutes depending on your XML file size.

In [9]:
print("\n" + "#"*60)
print("# DRUGBANK DATA EXTRACTION - STARTING")
print("#"*60)

# Extract all data
df_drugs, df_accessions, df_synonyms, df_salts = extract_drug_identifiers(XML_FILE)
df_drug_interactions, df_food_interactions = extract_interactions(XML_FILE)
df_categories = extract_categories(XML_FILE)
df_products, df_ingredients = extract_products_and_ingredients(XML_FILE)
df_toxicity = extract_toxicity_contraindications(XML_FILE)

print("\n" + "#"*60)
print("# EXTRACTION COMPLETE!")
print("#"*60)


############################################################
# DRUGBANK DATA EXTRACTION - STARTING
############################################################

EXTRACTING: Drug Identifiers


Parsing drugs: 1014328it [02:46, 6076.05it/s]



Cleaning and deduplicating...

✓ drugs.csv: 19,830 drugs
  - With UNII: 14,411
  - Without UNII: 5,419
✓ accession_numbers.csv: 25,702 records
✓ drug_synonyms.csv: 52,027 synonyms
✓ salts.csv: 2,960 salt forms

EXTRACTING: Drug & Food Interactions


Parsing interactions: 1014328it [02:53, 5862.31it/s]



Cleaning and deduplicating...

✓ drug_interactions.csv: 2,910,010 interactions
✓ food_interactions.csv: 2,549 interactions

EXTRACTING: Drug Categories


...


Cleaning and deduplicating...

✓ drug_categories.csv: 107,361 category assignments

EXTRACTING: Products & Ingredients


Parsing products: 1014328it [02:50, 5934.42it/s]



Cleaning and deduplicating...

✓ drug_products.csv: 448,529 products
✓ product_ingredients.csv: 444,484 ingredient relationships

EXTRACTING: Toxicity & Contraindications


Parsing toxicity: 1014328it [02:33, 6595.33it/s]



Cleaning and deduplicating...

✓ toxicity_contraindications.csv: 20,325 records
  - With toxicity info: 2,606

############################################################
# EXTRACTION COMPLETE!
############################################################


## 5. Final Summary

In [10]:
print("\n" + "="*60)
print("FINAL SUMMARY - All Clean Files")
print("="*60)
print(f"\nOutput directory: {OUTPUT_DIR.absolute()}\n")

files = [
    ('drugs.csv', df_drugs),
    ('accession_numbers.csv', df_accessions),
    ('drug_synonyms.csv', df_synonyms),
    ('salts.csv', df_salts),
    ('drug_interactions.csv', df_drug_interactions),
    ('food_interactions.csv', df_food_interactions),
    ('drug_categories.csv', df_categories),
    ('drug_products.csv', df_products),
    ('product_ingredients.csv', df_ingredients),
    ('toxicity_contraindications.csv', df_toxicity)
]

for i, (filename, df) in enumerate(files, 1):
    print(f"{i:2d}. {filename:40s} {len(df):>8,} rows")

print("\n" + "="*60)
print("✓ All files cleaned and ready for Step 2 (Normalization)")
print("="*60)


FINAL SUMMARY - All Clean Files

Output directory: /Users/jovannafernando/Desktop/Capstone/drugbank_tables

 1. drugs.csv                                  19,830 rows
 2. accession_numbers.csv                      25,702 rows
 3. drug_synonyms.csv                          52,027 rows
 4. salts.csv                                   2,960 rows
 5. drug_interactions.csv                    2,910,010 rows
 6. food_interactions.csv                       2,549 rows
 7. drug_categories.csv                       107,361 rows
 8. drug_products.csv                         448,529 rows
 9. product_ingredients.csv                   444,484 rows
10. toxicity_contraindications.csv             20,325 rows

✓ All files cleaned and ready for Step 2 (Normalization)


## 6. Quick Data Quality Check

In [11]:
print("\n" + "="*60)
print("DATA QUALITY CHECKS")
print("="*60)

# Check drugs table
print("\n1. Drugs Table:")
print(f"   Total drugs: {len(df_drugs):,}")
print(f"   Drugs with UNII: {df_drugs['unii'].notna().sum():,} ({df_drugs['unii'].notna().sum()/len(df_drugs)*100:.1f}%)")
print(f"   Duplicate IDs: {df_drugs['drugbank_id'].duplicated().sum()}")

# Check interactions
print("\n2. Drug Interactions:")
print(f"   Total interactions: {len(df_drug_interactions):,}")
print(f"   Missing descriptions: {df_drug_interactions['description'].isna().sum():,}")

# Sample interaction
if len(df_drug_interactions) > 0:
    sample = df_drug_interactions.iloc[0]
    print("\n3. Sample Interaction:")
    print(f"   Drug: {sample['drug_id']}")
    print(f"   Interacts with: {sample['interacting_drug_name']}")
    print(f"   Description: {sample['description'][:100]}..." if pd.notna(sample['description']) else "   No description")

print("\n" + "="*60)
print("✓ Quality checks complete")
print("="*60)


DATA QUALITY CHECKS

1. Drugs Table:
   Total drugs: 19,830
   Drugs with UNII: 14,411 (72.7%)
   Duplicate IDs: 0

2. Drug Interactions:
   Total interactions: 2,910,010
   Missing descriptions: 0

3. Sample Interaction:
   Drug: DB00001
   Interacts with: Apixaban
   Description: Apixaban may increase the anticoagulant activities of Lepirudin....

✓ Quality checks complete


## 7. Preview Data 

In [12]:
# Preview drugs
print("\n=== DRUGS TABLE ===")
display(df_drugs.head())

# Preview interactions
print("\n=== DRUG INTERACTIONS ===")
display(df_drug_interactions.head())


=== DRUGS TABLE ===


,drugbank_id,name,unii,cas_number,description,indication,type
0,DB01373,Calcium,None,None,None,None,None
1,DB00001,Lepirudin,None,None,None,None,None
2,DB01022,Phylloquinone,None,None,None,None,None
4,DB00002,Cetuximab,None,None,None,None,None
6,DB00003,Dornase alfa,953A26OA1Y,143831-71-4,Dornase alfa is a biosynthetic form of human d...,Used as adjunct therapy in the treatment of cy...,biotech



=== DRUG INTERACTIONS ===


,drug_id,interacting_drug_id,interacting_drug_name,description
0,DB00001,DB06605,Apixaban,Apixaban may increase the anticoagulant activi...
1,DB00001,DB06695,Dabigatran etexilate,Dabigatran etexilate may increase the anticoag...
2,DB00001,DB01254,Dasatinib,The risk or severity of bleeding and hemorrhag...
3,DB00001,DB01609,Deferasirox,The risk or severity of gastrointestinal bleed...
4,DB00001,DB01586,Ursodeoxycholic acid,The risk or severity of bleeding and bruising ...


## 8. Search Specific Drugs 

In [13]:
# Example: Search for Metformin
search_term = "metformin"

matches = df_drugs[df_drugs['name'].str.contains(search_term, case=False, na=False)]
print(f"\nFound {len(matches)} drugs matching '{search_term}':")
display(matches[['drugbank_id', 'name', 'indication']])

if len(matches) > 0:
    drug_id = matches.iloc[0]['drugbank_id']
    
    # Find interactions
    interactions = df_drug_interactions[df_drug_interactions['drug_id'] == drug_id]
    print(f"\n{matches.iloc[0]['name']} has {len(interactions)} drug interactions:")
    display(interactions.head())
    
    # Find food interactions
    food_int = df_food_interactions[df_food_interactions['drug_id'] == drug_id]
    print(f"\n{matches.iloc[0]['name']} has {len(food_int)} food interactions:")
    display(food_int)


Found 2 drugs matching 'metformin':


,drugbank_id,name,indication
279710,DB00331,Metformin,**Metformin immediate-release formulations**\r...
1007706,DB15181,Metformin C-11,None



Metformin has 1266 drug interactions:


,drug_id,interacting_drug_id,interacting_drug_name,description
220430,DB00331,DB00082,Pegvisomant,The risk or severity of hypoglycemia can be in...
220431,DB00331,DB00204,Dofetilide,The serum concentration of Dofetilide can be i...
220432,DB00331,DB00209,Trospium,The serum concentration of Trospium can be dec...
220433,DB00331,DB00243,Ranolazine,The serum concentration of Metformin can be in...
220434,DB00331,DB00273,Topiramate,The risk or severity of lactic acidosis can be...



Metformin has 2 food interactions:


,drug_id,food_interaction
203,DB00331,Avoid alcohol.
204,DB00331,Take with food. Food reduces irritation.
